In [411]:
import sympy as sp

import re
from sympy import symbols, Symbol, init_printing

from scipy.special import comb

# 1) Symbol を継承して _latex をオーバーライド
class DoubleIndexSymbol(Symbol):
    def _latex(self, printer):
        m = re.fullmatch(r'([A-Za-z]+)_(\d+)_(\d+)__(\d+)', self.name)
        if m:
            base, i, j, k = m.groups()
            return f"{base}_{{{i},{j}}}^{k}"
        return self.name

# 2) MathJax 表示を有効化
init_printing(use_latex='mathjax')

In [412]:
n = 3
d = 3

file_path = f'data/n{n}-d{d}.dat-s'

In [413]:
def encode_variable(n, t, i, j):
    return i + (n + 1) * j + (n + 1) * (n + 1) * t + 1

def decode_variable_index(var_idx):
    var_idx -= 1
    i = var_idx % (n + 1)
    var_idx //= (n + 1)
    j = var_idx % (n + 1)
    var_idx //= (n + 1)
    t = var_idx % (n + 1)
    return i, j, t

def get_variable_symbol(var_idx):
    if var_idx == 0:
        return 'x_0'
    i, j, t = decode_variable_index(var_idx)
    return f"x_{i}_{j}__{t}"

In [414]:
## validation

for i in range(n + 1):
    for j in range(n + 1):
        for t in range(n + 1):
            var_idx = encode_variable(n, t, i, j)
            assert (i, j, t) == decode_variable_index(var_idx)

for idx in range(1, (n + 1) ** 3):
    i, j, t = decode_variable_index(idx)
    assert idx == encode_variable(n, t, i, j)

In [415]:
with open(file_path) as f:
    line = f.readline()
    num_variables = int(line.split()[0])

    var_str = " ".join([get_variable_symbol(i) for i in range(num_variables + 1)])
    variables = sp.symbols(var_str, cls=DoubleIndexSymbol)

    line = f.readline()
    nblock = int(line.split()[0])

    line = f.readline()
    block_sizes = map(int, line.split())

    line = f.readline()
    objective = map(int, line.split())
    
    line = f.readline()

    matrices = [
        sp.Matrix(abs(s), abs(s), lambda i,j: 0) for s in block_sizes
    ]

    while line:
        var_idx, block_idx, row, col, val = map(int, line.split())
        if var_idx == 0:
            matrices[block_idx - 1][row - 1, col - 1] += (-1) * val
        else:
            matrices[block_idx - 1][row - 1, col - 1] += val * variables[var_idx]
        line = f.readline()

In [416]:
# symmetrize
for mat in matrices:
    rows, cols = mat.shape
    for row in range(rows):
        for col in range(cols):
            if row <= col:
                continue
            mat[row, col] = mat[col, row]

In [417]:
variables

(x₀, x⁰₀ ₀, x⁰₁ ₀, x⁰₂ ₀, x⁰₃ ₀, x⁰₀ ₁, x⁰₁ ₁, x⁰₂ ₁, x⁰₃ ₁, x⁰₀ ₂, x⁰₁ ₂, x⁰₂ ↪

↪  ₂, x⁰₃ ₂, x⁰₀ ₃, x⁰₁ ₃, x⁰₂ ₃, x⁰₃ ₃, x¹₀ ₀, x¹₁ ₀, x¹₂ ₀, x¹₃ ₀, x¹₀ ₁, x¹ ↪

↪ ₁ ₁, x¹₂ ₁, x¹₃ ₁, x¹₀ ₂, x¹₁ ₂, x¹₂ ₂, x¹₃ ₂, x¹₀ ₃, x¹₁ ₃, x¹₂ ₃, x¹₃ ₃, x ↪

↪ ²₀ ₀, x²₁ ₀, x²₂ ₀, x²₃ ₀, x²₀ ₁, x²₁ ₁, x²₂ ₁, x²₃ ₁, x²₀ ₂, x²₁ ₂, x²₂ ₂,  ↪

↪ x²₃ ₂, x²₀ ₃, x²₁ ₃, x²₂ ₃, x²₃ ₃, x³₀ ₀, x³₁ ₀, x³₂ ₀, x³₃ ₀, x³₀ ₁, x³₁ ₁, ↪

↪  x³₂ ₁, x³₃ ₁, x³₀ ₂, x³₁ ₂, x³₂ ₂, x³₃ ₂, x³₀ ₃, x³₁ ₃, x³₂ ₃, x³₃ ₃)

In [418]:
block_idx = 1

matrices[block_idx - 1]

⎡ x⁰₀ ₀        3⋅x⁰₁ ₀            3⋅x⁰₂ ₀        x⁰₃ ₀ ⎤
⎢                                                      ⎥
⎢3⋅x⁰₁ ₀  3⋅x⁰₀ ₀ + 6⋅x⁰₂ ₀  6⋅x⁰₁ ₀ + 3⋅x⁰₃ ₀  3⋅x⁰₂ ₀⎥
⎢                                                      ⎥
⎢3⋅x⁰₂ ₀  6⋅x⁰₁ ₀ + 3⋅x⁰₃ ₀  3⋅x⁰₀ ₀ + 6⋅x⁰₂ ₀  3⋅x⁰₁ ₀⎥
⎢                                                      ⎥
⎣ x⁰₃ ₀        3⋅x⁰₂ ₀            3⋅x⁰₁ ₀        x³₃ ₃ ⎦

In [419]:
def calc_sym_diff(x, y, z):
    """
    * x, y, z \in C \subset {0, 1}^n
    * i=|x \bigtriangleup y|, j=|x \bigtriangleup z|, t=|(x \bigtriangleup y) \cap (x \bigtriangleup z)|
    """
    i, j, t = 0, 0, 0
    for idx in range(len(x)):
        if x[idx] != y[idx]:
            i += 1
        if x[idx] != z[idx]:
            j += 1
        if x[idx] != y[idx] and x[idx] != z[idx]:
            t += 1
    return i, j, t

def lambda_to_x(lamb, code_size, t, i, j):
    denom = (code_size * comb(n, i-t, exact=True) * comb(n-i+t,j-t, exact=True) * comb(n-i-j+2*t, t, exact=True))
    if denom == 0:
        return 0
    return lamb // denom

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
/var/folders/6k/nrp954t51fq50vv_ch4k2wc00000gn/T/ipykernel_4836/4120383699.py:2: SyntaxWarning: invalid escape sequence '\i'
  """


In [420]:
C = [
    "000", "111"
]

# C = [
#     "1000", "0100", "0010", "1110",
#     "0001", "1101", "1011", "0111",
# ]

# C = ["10", "01"]

In [421]:
effs_lambda = [0 for _ in range(num_variables + 1)]

for x in C:
    for y in C:
        for z in C:
            i, j, t = calc_sym_diff(x, y, z)
            var_idx = encode_variable(n, t, i, j)
            effs_lambda[var_idx] += 1

effs = [0 for _ in range(num_variables + 1)]
for i in range(n + 1):
    for j in range(n + 1):
        for t in range(n + 1):
            var_idx = encode_variable(n, t, i, j)
            effs[var_idx] = lambda_to_x(effs_lambda[var_idx], len(C), t, i, j)

effs
            

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ↪

↪  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  ↪

↪ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [422]:
subs_map = {}
for i in range(1, num_variables + 1):
    subs_map[variables[i]] = effs[i]

print(subs_map)

subs_matrices = []
for mat in matrices:
    subs_matrices.append(mat.subs(subs_map))

subs_matrices[0]

{x_0_0__0: 1, x_1_0__0: 0, x_2_0__0: 0, x_3_0__0: 1, x_0_1__0: 0, x_1_1__0: 0, x_2_1__0: 0, x_3_1__0: 0, x_0_2__0: 0, x_1_2__0: 0, x_2_2__0: 0, x_3_2__0: 0, x_0_3__0: 1, x_1_3__0: 0, x_2_3__0: 0, x_3_3__0: 0, x_0_0__1: 0, x_1_0__1: 0, x_2_0__1: 0, x_3_0__1: 0, x_0_1__1: 0, x_1_1__1: 0, x_2_1__1: 0, x_3_1__1: 0, x_0_2__1: 0, x_1_2__1: 0, x_2_2__1: 0, x_3_2__1: 0, x_0_3__1: 0, x_1_3__1: 0, x_2_3__1: 0, x_3_3__1: 0, x_0_0__2: 0, x_1_0__2: 0, x_2_0__2: 0, x_3_0__2: 0, x_0_1__2: 0, x_1_1__2: 0, x_2_1__2: 0, x_3_1__2: 0, x_0_2__2: 0, x_1_2__2: 0, x_2_2__2: 0, x_3_2__2: 0, x_0_3__2: 0, x_1_3__2: 0, x_2_3__2: 0, x_3_3__2: 0, x_0_0__3: 0, x_1_0__3: 0, x_2_0__3: 0, x_3_0__3: 0, x_0_1__3: 0, x_1_1__3: 0, x_2_1__3: 0, x_3_1__3: 0, x_0_2__3: 0, x_1_2__3: 0, x_2_2__3: 0, x_3_2__3: 0, x_0_3__3: 0, x_1_3__3: 0, x_2_3__3: 0, x_3_3__3: 1}


⎡1  0  0  1⎤
⎢          ⎥
⎢0  3  3  0⎥
⎢          ⎥
⎢0  3  3  0⎥
⎢          ⎥
⎣1  0  0  1⎦

In [423]:
find_not_psd = False

for block_idx, mat in enumerate(subs_matrices):
    eigenvals = mat.eigenvals()
    is_psd = all(ev >= 0 for ev in eigenvals.keys())

    if not is_psd:
        print(f'block [{block_idx + 1}] is not positive semedefinite!')
        display(mat)
        display(matrices[block_idx])
        find_not_psd = True

if not find_not_psd:
    print("Correct!")

Correct!
